# Project - Twitter Trades

Group: Individual (Ying Chan)


Data Sources:
The data source will be Twitter tweets API in general. We will combine this data set with the stock price data obtained preferrably by the minute or 5-minute intervals. The stock return will be calculated and tagged against tweets.

Objective:
The ultimate objective is to create a natural language processing algorithm which will create feature extractions from tweets that help explain siginificant movement in stock prices. For example, certain tweets such as "price gouging" might affect a pharmaceutical drug stock. 

Evaluate:
The evaluation technique for this type of model is quite high given the nature of the project, which is to see if certain tweets will be able to predict future moves that are significant. For example, if a tweet relating to "drug price" against this stock (or even other stocks) that causes the model to predict a movement does indeed predict a movement. The magnitude of movement is not an important measure here, though the direction is important. Another metric that can be measured will be a model portfolio performance using this algorithm, whether it will "beat" a traditional risk free rate return.

Load Tweepy library and start collecting search terms based on stocks in the S&P500. We start with a specific stock $RDUS for testing purposes of the algorithm. 

In [22]:
import tweepy
import csv

consumer_key="LX38iNoplbbMid20arGtsivQh"
consumer_secret="w1KQHobzm77FFdVMHIzHJJPPaPrHdx53e2Dq8CnuOwh0yr7luD"

access_token="268119650-HPK3lM3WSCnDAvl8d7PDmxwcWcRDC1XUZYr77doh"
access_token_secret="tu4UNdMdCWh0E7AuduS3EI2xliPSUv1TXTqPCj3NuglFb"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

results = api.search(q="$RDUS", count=1000 , include_entities=True, lang="en", since_id=2016-10-01)

with open('tweets.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile)
    for result in results:
        writer.writerow([result.created_at, unicode(result.text).encode("utf-8") ])

Create process tweet function to make all the words lower case and convert any hastags into normal words.

In [23]:
import re

#Perform basic pre-processing of tweet to improve our NLP model
def preprocess_tweet(tweet):
    #Lower case all words
    tweet = tweet.lower()
    #Remove URL strings
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Remove double spacing
    tweet = re.sub('[\s]+', ' ', tweet)
    #Remove usernames
    tweet = re.sub('@[^\s]+','TWITTER_USER_NAME',tweet)
    #Remove hashtags
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = tweet.strip('\'"')
    #strip punctuation
    return tweet

Read CSV data from tweets and stocks to merge and append to dataframe.

In [101]:
import pandas
tweets = pandas.read_csv('tweets.csv', header=None)
stocks = pandas.read_csv('rdus.us.txt')

tweets.columns = ['timestamp', 'tweet']

stocks['Timestamp'] = stocks.Date.astype(str).str.cat(stocks.Time.astype(str), sep=' ')

#Convert to pandas date time format
stocks['Timestamp'] = pd.to_datetime(stocks['Timestamp']) 
tweets['timestamp'] = pd.to_datetime(tweets['timestamp']) 

stocks['PreviousClose'] = stocks.Close.shift(1)
stocks['Return'] = (stocks.Close/stocks.PreviousClose)-1

print stocks

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Create function to find the closest date and apply to existing panda dataframes to join the tweets data to the stock data. We keep the tweets data as the left dataframe as this is the data that is of interest. If there is no tweet, there is no need to observe stock data. 

In [42]:
def find_closest_date(timepoint, time_series, add_time_delta_column=True):
    deltas = np.abs(time_series - timepoint)
    idx_closest_date = np.argmin(deltas)
    res = {"closest_date": time_series.ix[idx_closest_date]}
    idx = ['closest_date']
    if add_time_delta_column:
        res["closest_delta"] = deltas[idx_closest_date]
        idx.append('closest_delta')
    return pd.Series(res, index=idx)


tweets[['closest_timestamp', 'diff']] = tweets.timestamp.apply(
                                          find_closest_date, args=[stocks.Timestamp])

combined = pd.merge(tweets, stocks, left_on=['closest_timestamp'], right_on=['Timestamp'])

print combined

             timestamp                                              tweet  \
0  2016-11-17 04:03:33  $RDUS Company Radius Health, Inc. just submite...   
1  2016-11-16 23:16:11  The seasonality trend for $RDUS might surprise...   
2  2016-11-16 17:28:27  Radius Health Inc. $RDUS Stake Boosted by Jenn...   
3  2016-11-16 08:04:25  Radius Health Inc. $RDUS Receives Average Reco...   
4  2016-11-15 22:35:19  $ALBO $RDUS $CPHI $STRP $ON \n\nBreakout/Algor...   
5  2016-11-15 21:22:58  RT @d_voted_1: $RDUS Can we take a breather?  ...   
6  2016-11-15 19:20:21  $RDUS Can we take a breather?  Incredible rebo...   
7  2016-11-15 18:34:47  $RDUS gorgeous Daily 200sma rotation back over...   
8  2016-11-15 17:53:19  RT @biotechnova: Perceptive New Buys: $CLCD $R...   
9  2016-11-15 17:50:54  RT @biotechnova: Perceptive New Buys: $CLCD $R...   
10 2016-11-15 17:49:34  Perceptive New Buys: $CLCD $RDUS $FLML $AERI, ...   
11 2016-11-15 17:48:56  @Lblegend33 @NguyenTu0319 @Steam_Sports @thefl...   

Create feature list using function below.

In [43]:
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer

In [48]:
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = 'english',
    max_features = 85
)

In [59]:
import nltk.data
tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')

corpus_data_features = vectorizer.fit_transform(combined.tweet.tolist())

corpus_data_features_nd = corpus_data_features.toarray()

In [60]:
vocab = vectorizer.get_feature_names()
print vocab

[u'abaloparatid', u'abx', u'acad', u'aeri', u'afternoon', u'alert', u'algorithm', u'amp', u'amzn', u'analyst', u'andybiotech', u'averag', u'axon', u'befor', u'bet', u'biib', u'biosimilar', u'buy', u'check', u'chmp', u'clcd', u'compani', u'dec', u'depo', u'di', u'dia', u'eem', u'ema', u'etsi', u'fb', u'flml', u'follow', u'forteo', u'friday', u'ftnw', u'galt', u'ge', u'googl', u'gt', u'gwph', u'health', u'http', u'ibb', u'impact', u'issu', u'jd', u'leak', u'mack', u'market', u'microcapcompani', u'mkt', u'mux', u'n', u'nbix', u'new', u'nke', u'open', u'opinion', u'oxi', u'place', u'posit', u'radiu', u'rdu', u'read', u'receiv', u'recommend', u'rt', u'runnin', u's', u'sell', u'signal', u'size', u'sleeper', u'sold', u'spb', u'stock', u't', u'thi', u'think', u'tomorrow', u'trade', u'tsla', u'w', u'x', u'z']


In [62]:
dist = np.sum(corpus_data_features_nd, axis=0)

for tag, count in zip(vocab, dist):
    print count, tag

7 abaloparatid
3 abx
3 acad
4 aeri
3 afternoon
6 alert
5 algorithm
3 amp
20 amzn
5 analyst
6 andybiotech
3 averag
3 axon
20 befor
3 bet
3 biib
7 biosimilar
16 buy
20 check
7 chmp
3 clcd
3 compani
3 dec
3 depo
20 di
20 dia
3 eem
7 ema
3 etsi
20 fb
3 flml
4 follow
7 forteo
3 friday
20 ftnw
20 galt
20 ge
20 googl
4 gt
4 gwph
14 health
49 http
3 ibb
7 impact
7 issu
20 jd
3 leak
3 mack
19 market
18 microcapcompani
7 mkt
3 mux
18 n
3 nbix
5 new
20 nke
19 open
7 opinion
3 oxi
3 place
8 posit
14 radiu
66 rdu
6 read
3 receiv
3 recommend
31 rt
10 runnin
12 s
6 sell
3 signal
7 size
3 sleeper
3 sold
3 spb
23 stock
44 t
8 thi
5 think
20 tomorrow
3 trade
20 tsla
3 w
3 x
4 z


In [97]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        corpus_data_features_nd[0:len(combined)], 
        combined.Return,
        train_size=0.85)

from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression()
log_model = log_model.fit(X=X_train, y=y_train)

In [98]:
y_test_final = pd.Series.tolist(y_test)
y_pred = log_model.predict(X_test)

print y_test_final
print y_pred



[0.0059928086296443528, 0.0031423290203327348, -0.0068130204390614013, 0.0031423290203327348, 0.0073786407766991413, -0.00079223608635370368, 0.0031423290203327348, 0.0, -0.0037206335707451466, 0.0019508388607101246, -0.013116679718089275, -0.013116679718089275, 0.0050260970423352536]
[-0.00292797  0.00314233 -0.00681302  0.00314233 -0.01284263 -0.00292797
  0.00314233 -0.00292797  0.02411438 -0.00236005  0.02411438 -0.00292797
 -0.00018215]
